# NEC winter-school: third exercise

In this notebook, we will learn how to finetune a model. To do that, we will take the model that we previously trained in MNIST and finetune it in Fashion-MNIST.

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from tensorflow import keras
import numpy as np                     # Basic array operation library
import matplotlib.pyplot as plt        # For plotting and displaying
%matplotlib inline

This time we want to train a model that works on Fashion-MNIST. So the first step is to prepare the dataset.

In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0         # Rescale the image values to [0,1], as we did before

# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

Display a grid of 5x5 digits:

In [3]:
# Add code


Print the 25 first labels:

In [4]:
# Fill me with code


Copy your class model, but replace your last layer.

In [5]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        # Use the class from the previous exercice


In [7]:
# Create an instance of the model
model = MyModel()

# Built the model
model.build((None,28,28,1))  # This is needed to initialize the shapes of the layers

Now we have to load the weights of the model that we previously trained.

In [ ]:
model.load_weights('mnist_model.h5')

Once we have the weights loaded, we are going to modify the output layer by randomly assigning new weights to it.

In [ ]:
model.d3.weights[0] = np.random.randn(2,10) * 0.02
model.d3.weights[1] = np.zeros((10))

Now we can compile the model and get ready to train it.

In [ ]:
model.compile(optimizer='adam',                         # Optimizer
              loss='sparse_categorical_crossentropy',   # Loss function
              metrics=['accuracy'])                     # Test metric to control that everything is working well

# Train
model.fit(x_train, y_train, epochs=5)

Let's save the model.

In [ ]:
model.save_weights('fashion-mnist_model.h5')

We test the model:

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

Try to train the model from scratch. Do you see any difference?

In [ ]:
model_scratch = MyModel()

model.compile(optimizer='adam',                         # Optimizer
              loss='sparse_categorical_crossentropy',   # Loss function
              metrics=['accuracy'])                     # Test metric to control that everything is working well

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)